# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd

from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
total_product_customer = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
total_product_customer

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [16]:
product_matrix = pd.pivot_table(data,values='Quantity',
                                index='ProductName',
                                columns='CustomerID',
                                aggfunc='sum').fillna(0)

product_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [33]:
squareform(pdist(product_matrix.T, metric='euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [88]:
customer_similarity = pd.DataFrame(1/(1 + squareform(pdist(product_matrix.T, metric='euclidean'))), 
                          index=product_matrix.columns, columns=product_matrix.columns)

round(customer_similarity, 3)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000,0.077,0.087,0.082,0.081,0.083,0.075,0.083,0.082,0.080,...,0.005,0.005,0.004,0.005,0.004,0.005,0.005,0.004,0.004,0.004
200,0.077,1.000,0.078,0.076,0.074,0.075,0.076,0.076,0.078,0.077,...,0.005,0.005,0.004,0.005,0.004,0.005,0.005,0.004,0.004,0.004
264,0.087,0.078,1.000,0.080,0.082,0.080,0.077,0.081,0.082,0.078,...,0.005,0.005,0.004,0.005,0.004,0.005,0.005,0.004,0.004,0.004
356,0.082,0.076,0.080,1.000,0.076,0.078,0.075,0.082,0.077,0.076,...,0.005,0.005,0.004,0.005,0.004,0.005,0.005,0.004,0.004,0.004
412,0.081,0.074,0.082,0.076,1.000,0.079,0.075,0.082,0.078,0.078,...,0.005,0.005,0.004,0.005,0.004,0.005,0.005,0.004,0.004,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.004,0.004,0.003,0.004,0.004,1.000,0.004,0.003,0.003,0.003
98069,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.004,0.004,0.003,0.004,0.003,0.004,1.000,0.003,0.003,0.003
98159,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,...,0.004,0.004,0.003,0.003,0.003,0.003,0.003,1.000,0.003,0.003


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [89]:
top5_customer = customer_similarity[412].sort_values(ascending=False)[:5]
top5_customer

CustomerID
412     1.000000
3317    0.085638
3535    0.084959
1008    0.083333
1072    0.082709
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [90]:
top5_customer.index.tolist()

[412, 3317, 3535, 1008, 1072]

In [91]:
similar_customers = total_product_customer[total_product_customer['CustomerID'].isin(top5_customer.index.tolist())]
similar_customers

,CustomerID,ProductName,Quantity
254,412,Apricots - Dried,1
255,412,Banana - Leaves,1
256,412,Bananas,1
257,412,Beans - Wax,1
258,412,Beef - Rib Eye Aaa,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [93]:
products_recom = pd.DataFrame(similar_customers.groupby('ProductName').sum()['Quantity'].sort_values(
                                                                ascending=False)).reset_index()
products_recom

,ProductName,Quantity
0,Spinach - Baby,3
1,Soup - Campbells Bean Medley,3
2,Curry Paste - Madras,3
3,"Pepper - Black, Whole",3
4,Soupfoamcont12oz 112con,3
...,...,...
210,Grouper - Fresh,1
211,Halibut - Fletches,1
212,Halibut - Steaks,1
213,Island Oasis - Mango Daiquiri,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [95]:
products_merge = products_recom.merge(product_matrix, on='ProductName')
products_merge = products_merge[products_merge[412] == 0]

products_left[:5]

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
1,Soup - Campbells Bean Medley,3,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
3,"Pepper - Black, Whole",3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
16,"Tart Shells - Sweet, 4",2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,Juice - Orange,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,25.0,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [110]:
recommendations = {}

for customer in data['CustomerID'].unique():
    top5_cust = customer_similarity[customer].sort_values(ascending=False).reset_index()[:5]
    similar_cust = total_product_customer[total_product_customer['CustomerID'].isin(top5_cust.index.tolist())]
    product_recomen = pd.DataFrame(similar_cust.groupby('ProductName').sum()['Quantity'].sort_values(
                                                                    ascending=False)).reset_index()
    product_merge = product_recomen.merge(product_matrix, on='ProductName')
    top_merge = products_merge['ProductName'][:5].values
    recommendations[customer] = top_merge

In [114]:
recommendations

{61288: array(['Soup - Campbells Bean Medley', 'Pepper - Black, Whole',
        'Wine - Blue Nun Qualitatswein', 'Tart Shells - Sweet, 4',
        'Juice - Orange'], dtype=object),
 77352: array(['Soup - Campbells Bean Medley', 'Pepper - Black, Whole',
        'Wine - Blue Nun Qualitatswein', 'Tart Shells - Sweet, 4',
        'Juice - Orange'], dtype=object),
 40094: array(['Soup - Campbells Bean Medley', 'Pepper - Black, Whole',
        'Wine - Blue Nun Qualitatswein', 'Tart Shells - Sweet, 4',
        'Juice - Orange'], dtype=object),
 23548: array(['Soup - Campbells Bean Medley', 'Pepper - Black, Whole',
        'Wine - Blue Nun Qualitatswein', 'Tart Shells - Sweet, 4',
        'Juice - Orange'], dtype=object),
 78981: array(['Soup - Campbells Bean Medley', 'Pepper - Black, Whole',
        'Wine - Blue Nun Qualitatswein', 'Tart Shells - Sweet, 4',
        'Juice - Orange'], dtype=object),
 83106: array(['Soup - Campbells Bean Medley', 'Pepper - Black, Whole',
        'Wine - Blue Nu

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [119]:
results_customer = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
results_customer.rename(columns={'index':'CustomerID',
                                0:'Recommendation1', 1:'Recommendation2', 2:'Recommendation3',
                                3:'Recommendation4', 4:'Recommendation5'})

,CustomerID,Recommendation1,Recommendation2,Recommendation3,Recommendation4,Recommendation5
0,61288,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
1,77352,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
2,40094,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
3,23548,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
4,78981,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
...,...,...,...,...,...,...
995,49005,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
996,41286,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
997,85878,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
998,68506,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [124]:
customer_similarity2 = pd.DataFrame(1/(1 + squareform(pdist(product_matrix.T, metric='cityblock'))), 
                          index=product_matrix.columns, columns=product_matrix.columns)

round(customer_similarity2, 3).head(5)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000,0.008,0.010,0.009,0.008,0.009,0.008,0.009,0.009,0.009,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.001
200,0.008,1.000,0.008,0.008,0.007,0.008,0.008,0.008,0.008,0.008,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.001
264,0.010,0.008,1.000,0.009,0.009,0.008,0.008,0.008,0.009,0.008,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.001
356,0.009,0.008,0.009,1.000,0.008,0.008,0.008,0.009,0.008,0.008,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.001
412,0.008,0.007,0.009,0.008,1.000,0.008,0.008,0.009,0.008,0.008,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.001


In [125]:
recommendations2 = {}

for customer in data['CustomerID'].unique():
    top5_cust = customer_similarity2[customer].sort_values(ascending=False).reset_index()[:5]
    similar_cust = total_product_customer[total_product_customer['CustomerID'].isin(top5_cust.index.tolist())]
    product_recomen = pd.DataFrame(similar_cust.groupby('ProductName').sum()['Quantity'].sort_values(
                                                                    ascending=False)).reset_index()
    product_merge = product_recomen.merge(product_matrix, on='ProductName')
    top_merge = products_merge['ProductName'][:5].values
    recommendations2[customer] = top_merge

In [126]:
results_customer2 = pd.DataFrame.from_dict(recommendations2, orient='index').reset_index()
results_customer2.rename(columns={'index':'CustomerID',
                                0:'Recommendation1', 1:'Recommendation2', 2:'Recommendation3',
                                3:'Recommendation4', 4:'Recommendation5'})

,CustomerID,Recommendation1,Recommendation2,Recommendation3,Recommendation4,Recommendation5
0,61288,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
1,77352,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
2,40094,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
3,23548,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
4,78981,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
...,...,...,...,...,...,...
995,49005,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
996,41286,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
997,85878,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
998,68506,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange


In [133]:
# comparison

# euclidean
results_customer.iloc[[10, 250, 590, 780, 999]]

,index,0,1,2,3,4
10,86595,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
250,96684,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
590,63803,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
780,31468,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
999,3472,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange


In [134]:
# cityblock

results_customer2.iloc[[10, 250, 590, 780, 999]]

,index,0,1,2,3,4
10,86595,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
250,96684,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
590,63803,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
780,31468,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
999,3472,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Juice - Orange
